In [2]:
using Pkg
Pkg.activate(".")
using MiniOps
using Random, LinearAlgebra

Random.seed!(0)

# Simple check
h  = [1.0, 2.0, -1.0]
A  = conv1d_cols_op(h)

nt, ntr = 50, 100
X  = randn(nt, ntr)                     # test input
Y  = A * X
Z  = randn(size(Y))                     # test output

ok, err = adjoint_test(A, X, Z)
@show ok err
 B = conv1d_op(ones(14))

e = opnorm_power(B, randn(9); niter=20)


ok = true

  Activating project at `~/MiniOps`



err = 1.8364041408825356e-16


9.98455513069699

In [4]:

result = linearity_test(B, randn(10), randn(10); ntests=3, tol=1e-10)
println(B.name, " Linearity test : " , result)

result = adjoint_test(B, randn(10), randn(10+14-1); tol=1e-10)
println(B.name, " Adjoint test   : ", result)

e = opnorm_power(B, randn(10); niter=20)
println(B.name, " Maximum Eig    : ", e)

Sc = scaling_op(3.0)
result = is_selfadjoint(Sc, randn(20); tol=1e-10)
println(Sc.name, " Self adjoint ? ", result)





conv1d Linearity test : (true, 3.513759946896693e-16)
conv1d Adjoint test   : (true, 1.5633845460063828e-16)
conv1d Maximum Eig    : 10.368948222129044
scaling Self adjoint ? (true, 0.0)


In [5]:
B = fft_op()

result = linearity_test(B, randn(128), randn(128); ntests=3, tol=1e-10)
println(B.name, " Linearity test : " , result)

result = adjoint_test(B, randn(128), randn(128); tol=1e-10)
println(B.name, " Adjoint test   : ", result)

e = opnorm_power(B, randn(ComplexF64,128); niter=10)
println(B.name, " Maximum Eig    : ", e)

fft Linearity test : (true, 2.1947560374829204e-16)
fft Adjoint test   : (true, 3.0112832686735434e-16)
fft Maximum Eig    : 11.313708498984761


In [6]:
 11.313708498984761^2

128.00000000000003